In [210]:
import math
import numpy as np
from scipy.stats import norm
from scipy.special import gammainc
from scipy.optimize import fsolve
from scipy.optimize import minimize
from scipy.stats import ncx2

In [211]:
def LowBiasSABR(F0, K, sigma0, alpha, beta, rho, T, T_step_1year, N, annuity):
    """ Low bias numerical scheme for SABR process. Algorithm from B. Chen, C.W. Osterlee. and H. Van Der Weide. (August 2011)

       Parameters
       ----------
       F0: Initial forward swap rate
       sigma0: Initial  volatility
       alpha: vol of vol
       beta: SABR Beta parameter
       rho: Spot vs. Volatility correlation
       T: Expiry of option (years)
       T_step_1year: number of steps per year
       N: Number of MC paths
       Annuity: swap annuity

       Returns
       -------
       Swaption price

    """


    return 0

In [212]:
F0 = 0.00
K = 0.00
sigma0 = 0.01 #initial vol
alpha=1.0 #vol of vol
beta=0.5
rho=-0.4
T=10.0
T_step_1year = 10
N=10000
annuity = 20.0
shift = 0.02

In [213]:
dt = 1. / T_step_1year
sqrtdt = np.sqrt(dt)
total_steps = int(T * T_step_1year)


In [214]:
tbeta = 2 * (1 - beta)

In [215]:
def tosolve(x, a, b, u):
    return 1 - ncx2.cdf(a, b, x) - u

In [216]:
def root_chi2(a, b, u):
    ''' inversion of the non central chi-square distribution '''
    c0 = a
    bnds = [(0., None)]
    res = minimize(equation, c0, args=(a, b, u), bounds=bnds)
    return res.x[0]

def equation(c, a, b, u):
    return 1 - ncx2.cdf(a, b, c) - u  # page 13 step 7 

In [217]:
Ft = np.full(N, F0 + shift)
sigmat = np.full(N, sigma0)
logsigmat = np.full(N, np.log(sigma0))
drift = - 0.5 * alpha ** 2 * sqrtdt

b = 2 - (1 - 2 * beta - rho**2 * (1 - beta)) / ((1 - beta) * (1 - rho ** 2))

for t in np.arange(dt, T+dt, dt):

    W2 = np.random.normal(0., sqrtdt, N)
    U1 = np.random.uniform(size=N)
    U = np.random.uniform(size=N)
    Z = np.random.normal(0., 1., N)

    sigmat0 = sigmat
    Ft0 = Ft

    #sigmat = sigmat0 * np.exp(alpha * W2 - 0.5 * alpha ** 2 * sqrtdt) #formula in paper
    logsigmat += alpha * W2 + drift 
    sigmat = np.exp(logsigmat)

    W2_2, W2_3, W2_4 = [np.power(W2, x) for x in [2,3,4]]
    m1 = alpha * W2
    m2 = (alpha ** 2) * (2 * W2_2 - dt / 2) / 3.
    m3 = (alpha ** 3) * (W2_3 - W2 * dt) / 3.
    m4 = (alpha ** 4) * (W2_4 * 2 / 3. - W2_2 * dt * 3 / 2. + 2 * dt ** 2) / 5.
    m = sigmat0 ** 2 * dt * (1. + m1 + m2 + m3 + m4)

    v = np.power(sigmat0, 4) * alpha ** 2 * np.power(dt, 3) / 3

    sigma2 = np.log(1 + v / (m ** 2))
    sigma = np.sqrt(sigma2)
    mu = np.log(m) - 0.5 * sigma2
    A = np.exp(sigma * norm.ppf(U1) + mu)
    nu = (1 - rho ** 2) * A

    a = (np.power(Ft0, 1-beta) / (1-beta) + rho * (sigmat - sigmat0) / alpha) ** 2 / nu

    prob = 1 - gammainc(b/2, a/2) #already normalised by Gamma(s)

    # Criterion for Quadratic Exponential Scheme
    k = 2 - b
    lambda_ = a
    m = k + lambda_
    s2 = 2 * (k + 2 * lambda_)

    psi = s2 / (m ** 2)
    twooverpsi = 2. / psi
    e2 = twooverpsi - 1. + np.sqrt(twooverpsi) * np.sqrt((twooverpsi) - 1.)
    d = m / (1. + e2)

    for n in range(0, N):

        if Ft0[n] < 1e-8 or prob[n] > U[n]:
            Ft[n] = 0.
            continue

        if psi[n] <= 1.5 and m[n] >= 0:
            Ft[n] = np.power(((1. - beta) ** 2) * nu[n] * d[n] * ((np.sqrt(e2[n]) + Z[n]) ** 2), 1. / tbeta) 
        else:
            cstar = fsolve(tosolve, a[n], args=(a[n], b, U[n]))
            Ft[n] = np.power(cstar * ((1. - beta) ** 2) * nu[n], 1. / (2. - 2. * beta))
            Ft[n] = 0

payoff = np.maximum(Ft - (K + shift), 0)
PV = annuity * np.mean(payoff)

print(PV)

0.01510138360600721


In [218]:
def hagan_implied_volatility_shifted(F, K, T, alpha, beta, rho, nu, shift):
    F_shifted = F + shift
    K_shifted = K + shift

    epsilon = 1e-07  # Small number to avoid division by zero
    if F_shifted <= 0 or K_shifted <= 0:
        raise ValueError("Shifted forward rate and strike must be positive.")

    if abs(F_shifted - K_shifted) < epsilon:
        # When F_shifted == K_shifted, use the ATM formula
        FK_beta = F_shifted ** (1 - beta)
        alpha_beta = alpha / FK_beta

        term1 = ((1 - beta) ** 2 / 24) * (alpha_beta) ** 2
        term2 = (rho * beta * nu * alpha) / (4 * FK_beta)
        term3 = (nu ** 2 * (2 - 3 * rho ** 2)) / 24

        sigma = alpha_beta * (1 + (term1 + term2 + term3) * T)
    else:
        logFK = np.log(F_shifted / K_shifted)
        FK_beta = (F_shifted * K_shifted) ** ((1 - beta) / 2)
        one_minus_beta = 1 - beta

        z = (nu / alpha) * FK_beta * logFK
        x_z = np.log((np.sqrt(1 - 2 * rho * z + z ** 2) + z - rho) / (1 - rho))

        # Avoid division by zero
        if abs(x_z) < epsilon:
            x_z = epsilon

        # Volatility formula
        numerator = alpha
        denominator = FK_beta * (1 + ((one_minus_beta ** 2 / 24) * (logFK) ** 2) +
                                 ((one_minus_beta ** 4 / 1920) * (logFK) ** 4))
        sigma = (numerator / denominator) * (z / x_z)

    return sigma

def black_scholes_price_shifted(F, K, T, sigma, annuity, shift):
    F_shifted = F + shift
    K_shifted = K + shift

    d1 = (np.log(F_shifted / K_shifted) + 0.5 * sigma ** 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    price = annuity * (F_shifted * norm.cdf(d1) - K_shifted * norm.cdf(d2))
    return price

In [219]:
sigma_hagan_shifted = hagan_implied_volatility_shifted(F0, K, T, sigma0, beta, rho, alpha, shift)
swaption_price_hagan_shifted = black_scholes_price_shifted(F0, K, T, sigma_hagan_shifted, annuity, shift)

print(swaption_price_hagan_shifted)

0.0567362210854663
